In [1]:
import os
from PIL import Image
from tqdm import tqdm

def center_crop_and_save(hazy_dir, gt_dir, out_hazy_dir, out_gt_dir, crop_size=(64, 64)):
    os.makedirs(out_hazy_dir, exist_ok=True)
    os.makedirs(out_gt_dir, exist_ok=True)

    valid_ext = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    hazy_files = [f for f in os.listdir(hazy_dir) if f.lower().endswith(valid_ext)]

    for fname in tqdm(hazy_files, desc="Cropping center patches"):
        hazy_path = os.path.join(hazy_dir, fname)
        gt_path = os.path.join(gt_dir, fname)

        if not os.path.exists(gt_path):
            print(f"⚠️ Missing GT for {fname}, skipping.")
            continue

        try:
            hazy_img = Image.open(hazy_path).convert('RGB')
            gt_img = Image.open(gt_path).convert('RGB')
        except:
            print(f"⚠️ Error reading {fname}, skipping.")
            continue

        w, h = hazy_img.size
        crop_w, crop_h = crop_size

        if w < crop_w or h < crop_h:
            print(f"⚠️ {fname} too small for cropping, skipping.")
            continue

        x = (w - crop_w) // 2
        y = (h - crop_h) // 2

        hazy_crop = hazy_img.crop((x, y, x + crop_w, y + crop_h))
        gt_crop = gt_img.crop((x, y, x + crop_w, y + crop_h))

        hazy_crop.save(os.path.join(out_hazy_dir, fname))
        gt_crop.save(os.path.join(out_gt_dir, fname))

# === Usage ===
train_data_dir = r"C:\Users\abd\d\ai\dehaze\dataset\RESIDE-6K\train"
test_data_dir = r"C:\Users\abd\d\ai\dehaze\dataset\RESIDE-6K\test"  # <-- adjust if needed
# train_data_dir = "/kaggle/input/reside6k/RESIDE-6K/train"
# val_data_dir = "/kaggle/input/reside6k/RESIDE-6K/val"  # <-- adjust if needed


out_dir_train = r"C:\Users\abd\d\ai\dehaze\dataset\reside_processed\kaggle\working\cropped_t"
out_dir_val = r"C:\Users\abd\d\ai\dehaze\dataset\reside_processed\kaggle\working\cropped_v"
crop_size = (128, 128)

# Train
center_crop_and_save(
    hazy_dir=f"{train_data_dir}/hazy",
    gt_dir=f"{train_data_dir}/GT",
    out_hazy_dir=f"{out_dir_train}/hazy",
    out_gt_dir=f"{out_dir_train}/GT",
    crop_size=crop_size
)

# Validation
center_crop_and_save(
    hazy_dir=f"{test_data_dir}/hazy",
    gt_dir=f"{test_data_dir}/GT",
    out_hazy_dir=f"{out_dir_val}/hazy",
    out_gt_dir=f"{out_dir_val}/GT",
    crop_size=crop_size
)


Cropping center patches: 100%|██████████| 1001/1001 [00:08<00:00, 112.00it/s]
